In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding 


In [3]:
df = pd.read_hdf('processed.h5', 'df')
df.head()

,review,sentiment,review_length,preprocessed_review,one_hot
0,One of the other reviewers has mentioned that ...,positive,307,"[[27, 2102, 1072, 147, 3184, 398, 3195, 203, 6...","[0.0, 1.0]"
1,A wonderful little production. <br /><br />The...,positive,162,"[[393, 120, 353, 1385, 2977, 17949, 1582, 411,...","[0.0, 1.0]"
2,I thought this was a wonderful way to spend ti...,positive,166,"[[190, 393, 95, 1155, 55, 879, 1494, 2657, 124...","[0.0, 1.0]"
3,Basically there's a family where a little boy ...,negative,138,"[[688, 18988, 236, 120, 420, 3639, 1251, 18988...","[1.0, 0.0]"
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,230,"[[6305, 112, 55, 290, 2141, 1427, 19, 103, 439...","[0.0, 1.0]"


In [39]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y_labels = encoder.fit_transform(df['sentiment'].values)
y_labels

array([1, 1, 1, ..., 0, 0, 0])

In [40]:
#splitting data 
x = np.array([review[0] for review in df['preprocessed_review']])
y = y_labels
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [41]:
#using a simple non deep learning model to predict the sentiment of a movie review
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(max_iter=1000)

lr_model.fit(x_train, y_train)

lr_predictions = lr_model.predict(x_test)
print("Logistic Regression Accuracy:", accuracy_score(y_test, lr_predictions))
print('Logistic Regression F1 score:', f1_score(y_test, lr_predictions))

Logistic Regression Accuracy: 0.5106
Logistic Regression F1 score: 0.5449135205504929


c:\Users\ctemi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [54]:
#Deep Learning model

#splitting the data
x = np.array([review[0] for review in df['preprocessed_review']])
y = np.array([label for label in df['one_hot']])

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

#calculating the max sentence length and vocabulary size
from helper_functions import preprocess_text
max_sentence_length = df.review.str.split().apply(len).max()
all_reviews = " ".join(df['review'])
a, tokenizer = preprocess_text(all_reviews, max_length=max_sentence_length)




#vocabulary size and max sentence length
vocab_size = len(tokenizer.word_index) + 1
max_sentence_length = x.shape[1]


214480